<a href="https://colab.research.google.com/github/RichardRossJr/Fairness-Scavenger-hunt/blob/main/fairlearn_Bank_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Fairness AI Microsoft
###Grace Nelson
5/2/2021

In [1]:
%matplotlib inline

In [2]:
pip install raiwidgets

     |████████████████████████████████| 3.9MB 22.7MB/s 
     |████████████████████████████████| 133kB 43.9MB/s 
     |████████████████████████████████| 788kB 38.3MB/s 
     |████████████████████████████████| 51kB 7.1MB/s 
     |████████████████████████████████| 5.5MB 43.5MB/s 
     |████████████████████████████████| 368kB 56.3MB/s 
     |████████████████████████████████| 256kB 55.4MB/s 
ERROR: sqlalchemy 1.4.15 has requirement greenlet!=0.4.17; python_version >= "3", but you'll have greenlet 0.4.17 which is incompatible.
ERROR: jupyter-console 5.2.0 has requirement prompt-toolkit<2.0.0,>=1.0.0, but you'll have prompt-toolkit 3.0.18 which is incompatible.
ERROR: google-colab 1.0.0 has requirement ipython~=5.5.0, but you'll have ipython 7.16.1 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
  Found existing installation: greenlet 1.1.0
    Uninstalling greenlet-1.1.0:
      Successfully uninstalled greenl

In [3]:
pip install fairlearn

     |████████████████████████████████| 24.6MB 120kB/s 


Fairlearn with Bank Marketing Data
===========================


This notebook shows how to use Fairlearn and the Fairness dashboard to
generate predictors for the Bank Marketing dataset. This dataset is a
classification problem - to predict if the client will subscribe (yes/no) a term deposit

We will first train a fairness-unaware predictor and show that it leads
to unfair decisions under a specific notion of fairness called
*demographic parity*.


Load and preprocess the data set
================================

We download the data set using fetch\_adult function in
fairlearn.datasets. We start by importing the various modules we're
going to use:

> **note**
>
> The `FairlearnDashboard`{.sourceCode} is no longer being developed as
> part of Fairlearn. The widget itself has been moved to [the raiwidgets
> package](https://pypi.org/project/raiwidgets/). Fairlearn will provide
> some of the existing functionality through
> `matplotlib`{.sourceCode}-based visualizations.


In [4]:
# from fairlearn.widget import FairlearnDashboard
from raiwidgets import FairnessDashboard
from sklearn.model_selection import train_test_split
from fairlearn.reductions import GridSearch
from fairlearn.reductions import DemographicParity, ErrorRate

from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
import pandas as pd

We can now load and inspect the data by using the fairlearn.datasets
module:


# Data Labels for the Dataset
1 - age (numeric)
2 - job : type of job (categorical: 'admin.','blue-collar','entrepreneur','housemaid','management','retired','self-employed','services','student','technician','unemployed','unknown')
3 - marital : marital status (categorical: 'divorced','married','single','unknown'; note: 'divorced' means divorced or widowed)
4 - education (categorical: 'basic.4y','basic.6y','basic.9y','high.school','illiterate','professional.course','university.degree','unknown')
5 - default: has credit in default? (categorical: 'no','yes','unknown')
6 - housing: has housing loan? (categorical: 'no','yes','unknown')
7 - loan: has personal loan? (categorical: 'no','yes','unknown')

related with the last contact of the current campaign:
8 - contact: contact communication type (categorical: 'cellular','telephone')
9 - month: last contact month of year (categorical: 'jan', 'feb', 'mar', ..., 'nov', 'dec')
10 - day_of_week: last contact day of the week (categorical: 'mon','tue','wed','thu','fri')
11 - duration: last contact duration, in seconds (numeric). Important note: this attribute highly affects the output target (e.g., if duration=0 then y='no'). Yet, the duration is not known before a call is performed. Also, after the end of the call y is obviously known. Thus, this input should only be included for benchmark purposes and should be discarded if the intention is to have a realistic predictive model.

other attributes:
12 - campaign: number of contacts performed during this campaign and for this client (numeric, includes last contact)
13 - pdays: number of days that passed by after the client was last contacted from a previous campaign (numeric; 999 means client was not previously contacted)
14 - previous: number of contacts performed before this campaign and for this client (numeric)
15 - poutcome: outcome of the previous marketing campaign (categorical: 'failure','nonexistent','success')

social and economic context attributes
16 - emp.var.rate: employment variation rate - quarterly indicator (numeric)
17 - cons.price.idx: consumer price index - monthly indicator (numeric)
18 - cons.conf.idx: consumer confidence index - monthly indicator (numeric)
19 - euribor3m: euribor 3 month rate - daily indicator (numeric)
20 - nr.employed: number of employees - quarterly indicator (numeric)

Output variable (desired target):
21 - y - has the client subscribed a term deposit? (binary: 'yes','no')

In [5]:
from sklearn.datasets import fetch_openml


data = fetch_openml(data_id=1461, as_frame=True)
X_raw = data.data
Y = (data.target)
X_raw

ValueError: ignored

We are going to treat the Level of Education of each individual as a sensitive feature and in this particular
case we are going separate this feature out and drop it from the main
data. We then perform some standard data preprocessing steps to convert
the data into a format suitable for the ML algorithms


Here we are treating level of education as sensitve

In [ ]:
A = X_raw["V4"]
X = X_raw.drop(labels=['V4'], axis=1)
X = pd.get_dummies(X)

sc = StandardScaler()
X_scaled = sc.fit_transform(X)
X_scaled = pd.DataFrame(X_scaled, columns=X.columns)

le = LabelEncoder()
Y = le.fit_transform(Y)

In [ ]:
X_scaled

Finally, we split the data into training and test sets:


In [ ]:
X_train, X_test, Y_train, Y_test, A_train, A_test = train_test_split(X_scaled,
                                                                     Y,
                                                                     A,
                                                                     test_size=0.2,
                                                                     random_state=0,
                                                                     stratify=Y)

# Work around indexing bug
X_train = X_train.reset_index(drop=True)
A_train = A_train.reset_index(drop=True)
X_test = X_test.reset_index(drop=True)
A_test = A_test.reset_index(drop=True)

Training a fairness-unaware predictor
=====================================

To show the effect of Fairlearn we will first train a standard ML
predictor that does not incorporate fairness. For speed of
demonstration, we use the simple
sklearn.linear\_model.LogisticRegression class:


In [ ]:
unmitigated_predictor = LogisticRegression(solver='liblinear', fit_intercept=True)

unmitigated_predictor.fit(X_train, Y_train)

We can load this predictor into the Fairness dashboard, and assess its
fairness:


In [ ]:
y_pred = unmitigated_predictor.predict(X_test)
FairnessDashboard(sensitive_features=A_test,
                   y_true=Y_test,
                    y_pred = y_pred)

#Performance Metric: Accuracy
#Fairness Metric #1: Demographic Parity Difference
3.37% (The maximum difference in selection rate, that is the fraction with the predicted label 1, between any two groups)
This means that there is at max a 3% difference in selection between the different levels of education, we want the difference to be 0 and 3.37% is pretty close. This means that there is not much of a difference in terms of pure selection rates between the education levels

#Fairness Metric #2: Demographic Parity Ratio
57.6% (the minimum ratio of selection rates, that is the fraction with)
The demographic parity ratio is at the minimin 57% whihc means that at minimum some education levels are selected to be successes twice as much as others, Specifcially unknown and tertary are selected twice as much as primary and secondary.

#Fairness Metric #3: Error Rate Difference
5.61% (the maximum difference between the error rates of any two groups)
in terms of error rate we can see that at max difference in arror rates of any of the educations levels if 5%, this is rather small so we can see in terms of error rate all education level peform relatively close to eachother.

#Fairness Metric #4: Minimum F-1 Score
42.2% (F1-score is the harmonic mean of precision and recall, The minimum F1-score of all groups.)
We want F-1 to be close to 1 since it is the ratio of precision and recall (ratios of ppostives and negative) a score of 1 shows our model is very accurate. Since this model's score is at a minimum .42 and .42 is closer to Zero than one we can see that this model is less than mediocre at worst. 

#Selection Rate
Looking at the difference in both selection and false postive and false negative rate we can see that people with unknown or tertiary education are selected at almost twice the rate of those with just primary or secondary education. This means that they are marked as successful at twice the rate. 

